## Building predictive models

In [3]:
import pandas as pd
import numpy as np
import os

### Import Data

In [4]:
# set the path of processed data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [5]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [15]:
# train_df.info()

In [31]:
# test_df.info()

### Data preparation

In [6]:
X = train_df.loc[:,'Age':].as_matrix().astype('float')
y = train_df['Survived'].ravel()

In [7]:
print X.shape, y.shape

(891L, 32L) (891L,)


In [8]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(712L, 32L) (712L,)
(179L, 32L) (179L,)


In [9]:
# average survival in train and test
print 'mean survival in train : {0:.3f}'.format(np.mean(y_train))
print 'mean survival in test : {0:.3f}'.format(np.mean(y_test))

mean survival in train : 0.383
mean survival in test : 0.385


#### check scikit version

In [10]:
import sklearn

In [11]:
sklearn.__version__

'0.19.1'

###  Baseline model

In [12]:
# import function
from sklearn.dummy import DummyClassifier

In [13]:
# create model
model_dummy = DummyClassifier(strategy='most_frequent', random_state=0)

In [14]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [15]:
print 'score for baseline model : {0:.2f}'.format(model_dummy.score(X_test, y_test))

score for baseline model : 0.61


In [16]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [17]:
# accuracy
print 'accuracy for baseline model : {0:.02f}'.format(accuracy_score(y_test, model_dummy.predict(X_test)))

accuracy for baseline model : 0.61


In [18]:
# confusion matrix
print 'confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test)))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [19]:
# precision and recall
print 'precision for baseline model : {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test)))
print 'recall for baseline model : {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test)))

precision for baseline model : 0.00
recall for baseline model : 0.00


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### First Kaggle Submission

In [20]:
# converting to the matrix
test_X = test_df.as_matrix().astype('float')

In [21]:
# get predictions
predictions = model_dummy.predict(test_X)

In [22]:
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [23]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [24]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [25]:
df_submission.to_csv(submission_file_path, index=False)

In [26]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.as_matrix().astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to the file
    df_submission.to_csv(submission_file_path, index=False)

In [27]:
# get submission file
get_submission_file(model_dummy, '01_dummy_csv')

### Logistic regression model

In [28]:
# import function
from sklearn.linear_model import LogisticRegression

In [29]:
# create model
model_lr_1 = LogisticRegression(random_state=0)

In [30]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
# evaluate model
print 'score for logistic regression version 1 : {0:.2f}'.format(model_lr_1.score(X_test, y_test))

score for logistic regression version 1 : 0.83


In [32]:
# performance metrics
# accuracy
print 'accuracy for logistic regression - version 1 : {0:.2f}'.format(accuracy_score(y_test, model_lr_1.predict(X_test)))
# confusion matrix
print 'confusion matrix for logistic regression - version 1 : \n {0}'.format(confusion_matrix(y_test, model_lr_1.predict(X_test)))
# precision
print 'precision for logistic regression - version 1 : {0:.2f}'.format(precision_score(y_test, model_lr_1.predict(X_test)))
# recall
print 'recall for logistic regression - version 1 : {0:.2f}'.format(recall_score(y_test, model_lr_1.predict(X_test)))

accuracy for logistic regression - version 1 : 0.83
confusion matrix for logistic regression - version 1 : 
 [[95 15]
 [15 54]]
precision for logistic regression - version 1 : 0.78
recall for logistic regression - version 1 : 0.78


In [33]:
# model coefficients
model_lr_1.coef_

array([[-0.0284227 ,  0.00455451, -0.50009091,  0.61781314, -0.8139233 ,
         0.12845079, -0.1728179 , -0.39317833,  0.52159976,  1.09941224,
         0.40341217, -0.18345051, -0.30036041,  0.96533486,  0.48256743,
        -0.34483447,  0.28089591,  1.21761326,  0.56363966, -1.44586303,
         1.07245555, -0.11273706, -0.47293647,  0.16255645,  0.24716927,
         0.28009442,  0.41324768,  0.49183529,  0.46198829,  0.14924424,
         0.37283518,  0.73023264]])

### Second kaggle submission

In [34]:
get_submission_file(model_lr_1, '02_lr.csv')

## Part 2

### Hyperparameter optimization

In [35]:
# base model
model_lr = LogisticRegression(random_state=0)

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
parameters = {'C':[1.0,10.0, 50.0, 100.0, 1000.0], 'penalty':['l1', 'l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)

In [38]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [39]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [40]:
print 'best score : {0:.2f}'.format(clf.best_score_)

best score : 0.83


In [41]:
# evaluate model 
print 'score for logistic regression - version 2 : {0:.2f}'.format(clf.score(X_test, y_test))

score for logistic regression - version 2 : 0.83


### Making third submission

In [42]:
# get submission file
get_submission_file(clf, '03_lr.csv')

### Feature Normalization and Standardization

In [43]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature Normalization

In [44]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [45]:
X_train_scaled[:,0].min(),X_train_scaled[:,0].max()

(0.0, 1.0)

In [46]:
X_test_scaled = scaler.transform(X_test)

#### Feature Standardization

In [47]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Create Model after standardization

In [48]:
# base model
model_lr = LogisticRegression(random_state=0)
parameters = {'C':[1.0,10.0, 50.0, 100.0, 1000.0], 'penalty':['l1', 'l2']}
clf = GridSearchCV(model_lr, param_grid=parameters, cv=3)
clf.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1.0, 10.0, 50.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [49]:
clf.best_score_

0.8132022471910112

In [50]:
# evaluate model
print 'Score for logistic regression - version 2: {0:.2f}'.format(clf.score(X_test_scaled, y_test))

Score for logistic regression - version 2: 0.84


### Model Persistance

In [51]:
# import pickle library
import pickle

In [52]:
# create the file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')